In [31]:
import numpy as np
from keras.utils import to_categorical
from keras.layers import Input, Dropout, Dense, GRU, LSTM
from keras.optimizers import Adam
from keras.models import Sequential

In [32]:
# labels
with open('PEMS_trainlabels.txt', 'r', encoding='utf-8') as f:
    raw_line = f.readlines()
    train_y = raw_line[0][1:-2].split(' ')

y_train = []
for label in train_y:
    y_train.append(int(label))
y_train = np.array(y_train) - 1
y_train = to_categorical(y_train, 7)

with open('PEMS_testlabels.txt', 'r', encoding='utf-8') as f:
    raw_line = f.readlines()
    test_y = raw_line[0][1:-2].split(' ')

y_test = []
for label in test_y:
    y_test.append(int(label))
y_test = np.array(y_test) - 1
y_test = to_categorical(y_test, 7)

In [33]:
# data
with open('PEMS_train.txt', 'r', encoding='utf-8') as f:
    train_x_lines = f.readlines()

x_train = []
for line in train_x_lines:
    splitted_line = line[1:-1].split(' ')
    values_list = []
    for s in splitted_line:
        try:
            s = float(s)
        except ValueError:
            if ']' not in s:
                s = s.split(';')
                values_list.append(float(s[0]))
                values_list.append(float(s[1]))
            else:
                s.rstrip(']')
                values_list.append(float(s[0]))
        else:
            values_list.append(float(s))
    x_train.append(values_list)

x_train = np.array(x_train)
x_train = x_train.reshape((267, 144, 963))

with open('PEMS_test.txt', 'r', encoding='utf-8') as f:
    test_x_lines = f.readlines()

x_test = []
for line in test_x_lines:
    splitted_line = line[1:-1].split(' ')
    values_list = []
    for s in splitted_line:
        try:
            s = float(s)
        except ValueError:
            if ']' not in s:
                s = s.split(';')
                values_list.append(float(s[0]))
                values_list.append(float(s[1]))
            else:
                s.rstrip(']')
                values_list.append(float(s[0]))
        else:
            values_list.append(float(s))
    x_test.append(values_list)

x_test = np.array(x_test)
x_test = x_test.reshape((173, 144, 963))

In [34]:
print('x_train.shape: ', x_train.shape)
print('y_train.shape: ', y_train.shape)
print('x_test.shape: ', x_test.shape)
print('y_test.shape: ', y_test.shape)

x_train.shape:  (267, 144, 963)
y_train.shape:  (267, 7)
x_test.shape:  (173, 144, 963)
y_test.shape:  (173, 7)


In [35]:
model = Sequential()
model.add(LSTM(128, input_shape=x_train[0].shape, return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(32, 'relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001))
model.fit(x_train, y_train, batch_size=32, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
9/9 [==============================] - 4s 174ms/step - loss: 1.9611 - val_loss: 1.9448
Epoch 2/100
9/9 [==============================] - 1s 106ms/step - loss: 1.9320 - val_loss: 1.9291
Epoch 3/100
9/9 [==============================] - 1s 111ms/step - loss: 1.9175 - val_loss: 1.9040
Epoch 4/100
9/9 [==============================] - 1s 107ms/step - loss: 1.8918 - val_loss: 1.8717
Epoch 5/100
9/9 [==============================] - 1s 110ms/step - loss: 1.8570 - val_loss: 1.8162
Epoch 6/100
9/9 [==============================] - 1s 110ms/step - loss: 1.7526 - val_loss: 1.7260
Epoch 7/100
9/9 [==============================] - 1s 107ms/step - loss: 1.8862 - val_loss: 2.2018
Epoch 8/100
9/9 [==============================] - 1s 106ms/step - loss: 2.0012 - val_loss: 1.8887
Epoch 9/100
9/9 [==============================] - 1s 107ms/step - loss: 1.8673 - val_loss: 1.8352
Epoch 10/100
9/9 [==============================] - 1s 105ms/step - loss: 1.7965 - val_loss: 1.7717
Epoch 11/

In [36]:
def model_accuracy(x, y, n=len(x_test)):
    true_cnt = 0
    for _ in range(n):
        rand_indx = np.random.randint(len(x_test))
        pred = np.argmax(model.predict(x_test[rand_indx:rand_indx + 1], verbose=False), axis=1)[0]
        true = np.argmax(y_test[rand_indx])
        if pred == true:
            true_cnt += 1
    print('\nModel accuracy =', true_cnt / n * 100, '%')


model_accuracy(x_test, y_test)


Model accuracy = 83.23699421965318 %


In [52]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']


def model_predict(test_indx):
    day_indx = model.predict(np.expand_dims(x_test[test_indx], axis=0), verbose=False)[0].argmax()
    print('Model prediction: ', days[day_indx])
    print('True: ', days[y_test[test_indx].argmax()])


model_predict(26)

Model prediction:  Friday
True:  Friday
